# Calibration for isolated runs

In [ ]:
from collections import OrderedDict
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import yt

%matplotlib inline
yt.mylog.setLevel(40)

## Star formation history

In [ ]:
# Specify simulations
sims = OrderedDict([
    ('isolated-common-old', 'Old Common Feedback'),
    ('isolated-common', 'Common Feedback'),
    ('isolated-favorite', 'Favorite Feedback')
])

In [ ]:
# Collect data
sfh = {}
for name in sims.keys():
    print(name)
    sfh_file = Path(f'cache/sfh-{name}.csv')
    if not sfh_file.exists():   
        # Make SFH table
        output_dir = Path(f'data/sim/{name}/output')
        rows = []
        for snap in sorted(Path(output_dir).glob('*.hdf5')):
            ds = yt.load(str(snap), bounding_box=[[-1e3, 1e3]]*3)
            ad = ds.all_data()
            row = pd.Series()
            row['t'] = float(ds.current_time.to('Gyr'))
            try:
                row['m_star'] = float(
                    ad['PartType4', 'particle_mass'].sum().to('Msun')
                )
            except:
                row['m_star'] = 0
            rows.append(row)
        df = pd.DataFrame(rows)
        sfh_file.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(sfh_file)
    # Read SFH table
    sfh[name] = pd.read_csv(sfh_file, index_col=0)

In [ ]:
# Make plot
fig, ax = plt.subplots()
ax.axhline(1e9, color='grey')
for name, label in sims.items():
    df = sfh[name]
    ax.plot(df.t, df.m_star, label=label)
ax.legend()
ax.set_xlabel('Time [$Gyr$]')
ax.set_ylabel('Stellar Mass [$M_\odot$]')
fig.savefig('cache/sfh-isolated.pdf', dpi=200)